# Automated stock recommendation tool

based on Value Rank stock rating method.

First Part of the code is for getting all the neccessary data from Finviz for each of the stocks that is given by using certain search criteria

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
headers = {'User-Agent': 'Mozilla/5.0'}
part="&r={}"
pageslist=['1','2','6']
new1=[]
new2=[]
new3=[]
tlist2=[]
tlist3=[]
for k in pageslist:
    url = "http://www.finviz.com/screener.ashx?v=1{}1&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe".format(k)
#find number of pages on the site if cant find then its 1
    page=requests.get(url, headers=headers)
    soup=BeautifulSoup(page.content, 'html.parser')
    table=soup.find(id="screener-content")
    
    if k=='2':
        titles=table.find_all(class_="table-top")
        #print(titles)
        for td in titles:
            text = td.find(text=True)
            tlist2.append(text)
    
    
    if k=='6':
        titles=table.find_all(class_="table-top")
        for td in titles:
            text = td.find(text=True)
            tlist3.append(text)

    try:
        pagelist=[]
        scnrclass=table.find_all(class_="screener-pages")
        for i in range(len(scnrclass)):
            #print(i)
            numero=scnrclass[i]
            pagelist.append(numero.get_text())
        pagelist = list(map(int, pagelist))
        #print(pagelist)
        a=max(pagelist)
    except:
        a=1
    #print (a)
    

    i=0
    for a in range(a):
        time.sleep( 2 )
        try:
            if i == 0:
                url1 = url
            
                print(url1)
                i=21
            else:
                url1 = (url+part).format(i)
                print(url1)
                i += 20
      
            page=requests.get(url1, headers=headers)

            soup=BeautifulSoup(page.content, 'html.parser')
            table=soup.find(id="screener-content")

            rows=["table-dark-row-cp", "table-light-row-cp"]
            for r in rows:
                row=table.find_all(class_=r)
                for tr in range(len(row)):
                    data=row[tr]
                    cols=data.find_all('td')
                    for td in cols:
                        text = td.find(text=True)
                        if k=='1':
                            new1.append(text)
                        elif k=='2':
                            new2.append(text)
                        elif k=='6':
                            new3.append(text)
        except:
            break
list1=[]
for i in range(11):
    list1.append(new1[i::11])
g=pd.DataFrame(list1).transpose()
g.columns=['No.', 'Ticker', 'Company', 
           'Sector', 'Industry', 
           'Country', 'Market Cap','P/E', 'Price', 'Change(pct)', 'Volume']
g['No.']=g['No.'].astype(int) #need to change stock number in order to sort it by index
g=g.sort_values(by='No.') #sort it first so its not 1-3-2-4
g_indexed=g.set_index(['No.']) #then set those as index
g_indexed['Price']=g_indexed['Price'].astype(float) #convert price to float
#g_indexed=g_indexed.sort_values(by='Price', ascending=False)
g_indexed['Volume']=g_indexed['Volume'].str.replace(',','').astype(int) #convert volume to float
g_indexed=g_indexed.replace('-',np.NaN) #this fills all cells where there is '-' with NaN (maybe better for other ops later)
g_indexed['P/E']=g_indexed['P/E'].astype(float) #this is for P/E
g_indexed['Change(pct)']=g_indexed['Change(pct)'].str.strip('%').astype(float) #strips percentage sign and converts to float (easier to manipulate)

#to convert market cap into float vlaue (so without Bs and Ms)
d = {'M': 10**6, 'B': 10**9}
k = '|'.join(d.keys())
a = g_indexed['Market Cap'].replace(d, regex=True).astype(float)
b = g_indexed['Market Cap'].replace([k], '', regex=True).astype(float)
s = a.mul(b)
#s = a.mul(b).sort_values(ascending=False)    #use THIS is if we want to sort by market cap here
#g_indexed.reindex(s.index))    

g_indexed['Market Cap']=s


tlist2.insert(3, 'P/E')
list2=[]
for i in range(18):
    list2.append(new2[i::18])
v=pd.DataFrame(list2).transpose()
v.columns=tlist2
v['No.']=v['No.'].astype(int) #need to change stock number in order to sort it by index
v=v.sort_values(by='No.') #sort it first so its not 1-3-2-4
v_indexed=v.set_index(['No.'])



            
list3=[]
for i in range(18):
    list3.append(new3[i::18])
f=pd.DataFrame(list3).transpose()
f.columns=tlist3
f['No.']=f['No.'].astype(int) #need to change stock number in order to sort it by index
f=f.sort_values(by='No.') #sort it first so its not 1-3-2-4
f_indexed=f.set_index(['No.'])

http://www.finviz.com/screener.ashx?v=111&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe
http://www.finviz.com/screener.ashx?v=111&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe&r=21
http://www.finviz.com/screener.ashx?v=121&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe
http://www.finviz.com/screener.ashx?v=121&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe&r=21
http://www.finviz.com/screener.ashx?v=161&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe
http://www.finviz.com/screener.ashx?v=161&f=cap_midunder,ind_gold,sec_basicmaterials&o=-pe&r=21


As the scraper returned too much data, I create new dataframe which consist of 5 important indicators that are needed for 1st step of composite value rank. 
This method is described here: http://help.stockopedia.com/product-guide/stockranks/basics/the-value-rank
All the missing values are filled wih 100000, which is highest as possible number or P/E, P/S, P/B, P/FCF since we wont be taking them into account anymore, and for dividend the worst value is 0 therefore 0 are given for those with missing values

In [2]:
masterdf=pd.concat([v_indexed['Ticker'],v_indexed['P/E'],v_indexed['P/S'], v_indexed['P/B'], v_indexed['P/FCF'], f_indexed['Dividend']], axis=1)
masterdf['Dividend']=masterdf['Dividend'].replace('-',0) #1st clean from '-'
masterdf=masterdf.replace('-',100000)
nums = masterdf.drop('Ticker', 1)
nums['Dividend']=nums['Dividend'].str.strip('%').astype(float) #or do it in one life when creating nums with assign like below
nums = nums.apply(pd.to_numeric, errors='coerce')
nums = nums.assign(Dividend=nums.Dividend / 100)
masterdf=masterdf[['Ticker']].join(nums)
masterdf['Dividend']=masterdf['Dividend'].replace('NaN',0)
masterdf.sort_values(by='P/S', ascending=True)

masterdf

,Ticker,P/E,P/S,P/B,P/FCF,Dividend
No.,,,,,,
1,TAHO,5090.00,2.56,0.61,100000.00,0.0472
2,AKG,545.00,1.05,0.51,72.44,NaN
3,SAND,76.17,10.17,1.28,100000.00,NaN
4,AGI,70.61,2.91,0.84,100000.00,0.0035
5,MUX,49.77,7.87,1.43,100000.00,0.0046
6,BTG,44.83,3.04,1.64,100000.00,NaN
7,GORO,31.24,3.32,3.31,30.59,0.0029
8,SSRM,29.34,2.84,1.27,13.57,NaN
9,GOLD,27.19,5.65,1.92,37.23,0.0261


After creating main datafrime now I rank all of the columns and give every row a specific value based on its ranking.
The ranking formula I used is: 


$$y = 100*(1-\frac{(x-1)}{maxvalue})$$

And the max value is max for each of the columns(indicators)

Then create new column and sum all the values and get total for each stock

In [3]:
rankeddf=pd.concat([masterdf.Ticker,
           masterdf.filter(regex="^P").rank(method="dense"),
           masterdf.Dividend.rank(method="dense", ascending=False)], 
          axis=1)

def calcrank(x):
    x.iloc[:, 1:] =  100*(1-(x.iloc[:, 1:] - 1).div(rankeddf.iloc[:, 1:].max()))
    return x

calcrank(rankeddf)
rankeddf['sumfirst']=rankeddf.sum(axis=1)

rankeddf=rankeddf.sort_values(by='sumfirst',ascending=False)

rankeddf


,Ticker,P/E,P/S,P/B,P/FCF,Dividend,sumfirst
No.,,,,,,,
16,KGJI,100.000000,100.000000,100.000000,100.000000,NaN,400.000000
11,DRD,70.588235,87.096774,60.526316,18.181818,93.333333,329.726477
24,SBGL,5.882353,96.774194,78.947368,9.090909,100.000000,290.694824
15,IAG,94.117647,54.838710,68.421053,63.636364,NaN,281.013773
13,ALO,82.352941,93.548387,94.736842,9.090909,NaN,279.729079
34,HMY,5.882353,90.322581,92.105263,9.090909,80.000000,277.401106
35,AU,5.882353,83.870968,55.263158,72.727273,46.666667,264.410418
12,KGC,76.470588,70.967742,65.789474,36.363636,NaN,249.591440
8,SSRM,52.941176,45.161290,52.631579,90.909091,NaN,241.643137


Now I create new dataframe that will contain stocks for futher data acquisition from Yahoo Finance.
It take best stocks that can reach potentially highest total score out of all in this ranking (highest total possible is 600)
The end result is to choose top 10% stocks based on their sum total

In [4]:
top10=round(len(rankeddf)*0.1) #get top 10% rounded
getmin=rankeddf['sumfirst'].head(top10).min() #then get min value from that
treshold=getmin-100 #caluclate threshold so that can be put into list for later scraping from yahoo
finaldf=rankeddf.loc[rankeddf['sumfirst'] > treshold] #get potentail list for yahoo scraper using cutoff equation
finaldf

,Ticker,P/E,P/S,P/B,P/FCF,Dividend,sumfirst
No.,,,,,,,
16,KGJI,100.000000,100.000000,100.000000,100.000000,NaN,400.000000
11,DRD,70.588235,87.096774,60.526316,18.181818,93.333333,329.726477
24,SBGL,5.882353,96.774194,78.947368,9.090909,100.000000,290.694824
15,IAG,94.117647,54.838710,68.421053,63.636364,NaN,281.013773
13,ALO,82.352941,93.548387,94.736842,9.090909,NaN,279.729079
34,HMY,5.882353,90.322581,92.105263,9.090909,80.000000,277.401106
35,AU,5.882353,83.870968,55.263158,72.727273,46.666667,264.410418
12,KGC,76.470588,70.967742,65.789474,36.363636,NaN,249.591440
8,SSRM,52.941176,45.161290,52.631579,90.909091,NaN,241.643137


Now scrape missing data (ev/bitda) for the stocks above from Yahoo Finance

In [14]:
yahoolist=finaldf['Ticker'].tolist()
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
option = webdriver.ChromeOptions()
#option.add_argument("--incognito")
browser = webdriver.Chrome(executable_path='C:/Users/Admin/chromedriver/chromedriver', chrome_options=option)

rval=[]
stockname=yahoolist
for i in stockname:   
    browser.get("https://finance.yahoo.com/quote/{}/key-statistics?p={}".format(i, i))
    timeout = 10
    WebDriverWait(browser, timeout)
    if i == stockname[0]:
        browser.find_element_by_xpath("//input[@type='submit' and @value='OK']").click()
        values_element = browser.find_elements_by_xpath("//td[@class='Fz(s) Fw(500) Ta(end)']")
        values = [x.text for x in values_element]
        rval.append(values[8])
    else:
        values_element = browser.find_elements_by_xpath("//td[@class='Fz(s) Fw(500) Ta(end)']")
        values = [x.text for x in values_element]
        rval.append(values[8])
    
for title, value in zip(stockname, rval):
    print(title + ': ' + value)
    
evdict=dict(zip(stockname, rval))



KGJI: 10.98
DRD: 6.50
SBGL: 5.77
IAG: 2.55
ALO: 1.34
HMY: 2.63
AU: 4.01
KGC: 4.11
SSRM: 5.68
TAHO: 8.26
GFI: 3.46
GOLD: 10.97
AKG: 3.15
AUY: 7.56


Create final dataframe and calulcate new sums with all 6 indicators. Also create new columns for different analysis
(for example if too much data is missing and we dont want to be biased)

In [15]:
evdf=pd.DataFrame(list(evdict.items()), columns=['Ticker', 'EV/EBITDA'])
evdf=evdf.replace('N/A',100000)
evdf['EV/EBITDA']=evdf['EV/EBITDA'].astype(float)
evdf['EV/EBITDA']=evdf['EV/EBITDA'].rank(method="dense")
def calcrank2(x):
    cols = evdf.select_dtypes([float,int])
    x[cols.columns] =  100*(1-(cols - 1).div(cols.max()))
    return x

lastdf=pd.concat([finaldf.reset_index(),evdf['EV/EBITDA']], axis=1)
lastdf['total']=lastdf['sumfirst']+lastdf['EV/EBITDA']
#lastdf.sort_values(by='total', ascending=False)
lastdf['nodiv']=lastdf['P/E']+lastdf['P/S']+lastdf['P/B']+lastdf['P/FCF']
#lastdf.sort_values(by='nodiv', ascending=False) #when too much div and ev ebitda are missing can look also at sum without those
lastdf['avgscore']=(lastdf.total/600+lastdf.nodiv/400)/2
lastdf.sort_values(by='avgscore', ascending=False) #average out the 

,No.,Ticker,P/E,P/S,P/B,P/FCF,Dividend,sumfirst,EV/EBITDA,total,nodiv,avgscore
0,16,KGJI,100.000000,100.000000,100.000000,100.000000,NaN,400.000000,4.0,404.000000,400.000000,0.836667
4,13,ALO,82.352941,93.548387,94.736842,9.090909,NaN,279.729079,12.0,291.729079,279.729079,0.592769
3,15,IAG,94.117647,54.838710,68.421053,63.636364,NaN,281.013773,3.0,284.013773,281.013773,0.587945
1,11,DRD,70.588235,87.096774,60.526316,18.181818,93.333333,329.726477,9.0,338.726477,236.393143,0.577763
7,12,KGC,76.470588,70.967742,65.789474,36.363636,NaN,249.591440,1.0,250.591440,249.591440,0.520816
8,8,SSRM,52.941176,45.161290,52.631579,90.909091,NaN,241.643137,11.0,252.643137,241.643137,0.512590
6,35,AU,5.882353,83.870968,55.263158,72.727273,46.666667,264.410418,6.0,270.410418,217.743751,0.497522
2,24,SBGL,5.882353,96.774194,78.947368,9.090909,100.000000,290.694824,2.0,292.694824,190.694824,0.482281
5,34,HMY,5.882353,90.322581,92.105263,9.090909,80.000000,277.401106,5.0,282.401106,197.401106,0.482086
12,2,AKG,17.647059,77.419355,89.473684,27.272727,NaN,211.812825,13.0,224.812825,211.812825,0.452110


Based on score the following stocks are worth considering: KGJI, ALO, IAG